In [1]:
import scCube
from scCube import scCube
from scCube.visualization import *
from scCube.utils import *
from matplotlib.pyplot import rc_context
import pandas as pd
import scanpy as sc
import numpy as np
import warnings
import time
warnings.filterwarnings("ignore")

In [2]:
model = scCube()

In [3]:
import os
new_directory = "/slurm/home/yrd/liaolab/chenrenjie/R_dataset/"
os.chdir(new_directory)

In [4]:
train_sample = '151673'
train_data = pd.read_csv(f"./DLPFC/{train_sample}/data.csv", index_col = 0)
train_meta = pd.read_csv(f"./DLPFC/{train_sample}/meta.csv", index_col = 0)
train_adata = model.pre_process(
    sc_data=train_data.T, 
    sc_meta=train_meta,
    is_normalized=False
    )
train_adata.obs['spot'] = train_adata.obs.index
train_adata.layers['data'] = train_adata.X

the input is count matrix, normalizing it firstly...


In [5]:
spatial_sample = '151674'
sp_data = pd.read_csv(f"./DLPFC/{spatial_sample}/data.csv", index_col = 0)
sp_meta = pd.read_csv(f"./DLPFC/{spatial_sample}/meta.csv", index_col = 0)
sp_adata = model.pre_process(
    sc_data=sp_data.T, 
    sc_meta=sp_meta,
    is_normalized=False
    )
sp_adata.obs['spot'] = sp_adata.obs.index
sp_adata.layers['data'] = sp_adata.X

the input is count matrix, normalizing it firstly...


In [10]:
save_path = "/slurm/home/yrd/liaolab/chenrenjie/Benchmark/Simulations/"
os.chdir(save_path)

In [11]:
generate_sc_meta, generate_sc_data = model.load_vae_and_generate_cell(
    sc_adata=train_adata,
    celltype_key='ground_truth',
    cell_key='spot',
    target_num=dict(sp_adata.obs.ground_truth.value_counts()),
    hidden_size=128,
    load_path="./DLPFC_151673train.pth",
    used_device='cuda:0')
generate_sc_data, generate_sc_meta = model.generate_pattern_reference(
        sc_adata=sp_adata,
        generate_sc_data=generate_sc_data,
        generate_sc_meta=generate_sc_meta,
        celltype_key='ground_truth',
        spatial_key=['imagecol', 'imagerow'],
        cost_metric='sqeuclidean'
)
generate_sc_data.to_csv(f"{save_path}/data.csv")
generate_sc_meta.to_csv(f"{save_path}/meta.csv")

generating by the targeted proportion of cell types...
loading model from ./DLPFC_151673train.pth
vae loading done!


Generate Epoch: 1: 100%|██████████| 3635/3635 [00:01<00:00, 2090.11it/s]


generated done!
data have been prepared!
